In [194]:
db_id = ''
db_pw = ''
db_nm = ''

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
import urllib
from tqdm import tqdm
import sys
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json
from IPython.display import display

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from lxml.html import fromstring
from bs4 import BeautifulSoup


# DB관련 함수정의
def db_connection(host_name=host_name):
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', cursorclass=pymysql.cursors.DictCursor) # cursorclass=pymysql.cursors.DictCursor 추가     # db=db,
    return conn


def engine_upload(upload_df: object, host_name: object = 'ds', db_name: object = db_nm, table_name: object = 'ranking') -> object:
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    upload_df.to_sql(table_name, engine_conn, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()

# 장르별 세부 카테고리 링크 수집

In [110]:
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

drv.get('https://www.goodreads.com/')
time.sleep(1)
soup = BeautifulSoup(drv.page_source, 'html.parser')

gr_list = soup.find('div', {'id':'browseBox'}).find_all('a',  {'class':'gr-hyperlink'})

# 모든 장르별 리스트 페이지 수집
gr_href = []
gr_name_list = []
list_href = []

for i in range(len(gr_list[:-1])):
    gr_href.append('https://www.goodreads.com/'+gr_list[i]['href'])
    gr_name = gr_list[i]['href'].split('/')[-1]
    gr_name_list.append(gr_name)
    list_href.append('https://www.goodreads.com/list/tag/'+gr_name)

In [168]:
# 각 장르별 리스트 페이지로 이동하여 세부 카테고리 수집
goodreads = pd.DataFrame(columns = ['genre', 'category_list', 'href', 'count'])

for h in range(0, len(list_href)):
    drv.get(list_href[h])
    time.sleep(1)
    
    # login popup
    try:
        drv.find_elements(By.CLASS_NAME, 'modal__close')[-1].click()
    except:
        pass
    
    status = True
    while status == True:
        soup = BeautifulSoup(drv.page_source, 'html.parser')

        # 장르별 리스트 접근, 한페이지에 최대 30개
        listopia = soup.find('div', {'class':'listRowsFull'}).find_all('div', {'class':'cell'})

        # 해당 리스트에 속하는 개별 타이틀 접근, 포함되는 도서 개수 수집
        cat_href = []
        cat_name = []
        cat_cnt = []

        for i in range(len(listopia)):
            cat_href.append('https://www.goodreads.com'+listopia[i].find('a', {'class':'listTitle'})['href'])
            cat_name.append(listopia[i].find('a', {'class':'listTitle'}).text)
            cnt = listopia[i].find('div', {'class':'listFullDetails'}).text.strip().replace('\n', '').split('book')[0]
            cat_cnt.append(int(cnt.replace(',', '').strip()))

        gr_data = [gr_name_list[h] for g in range(len(cat_href))]
        data = pd.DataFrame({'genre':gr_data, 'category_list':cat_name, 
                             'href':cat_href, 'count':cat_cnt})
        goodreads = pd.concat([goodreads, data], ignore_index=True)

        try:
            if drv.find_element(By.CLASS_NAME, 'next_page.disabled').text == 'next »':
                status = False
                print(f'{gr_name_list[h]} finished~!')
        except:
            drv.find_element(By.CLASS_NAME, 'next_page').click()
            time.sleep(2)

memoir finished~!
music finished~!
mystery finished~!
non-fiction finished~!
poetry finished~!
psychology finished~!
romance finished~!
science finished~!
science-fiction finished~!
self-help finished~!
sports finished~!
thriller finished~!
travel finished~!


AttributeError: 'NoneType' object has no attribute 'find_all'

In [169]:
goodreads

,genre,category_list,href,count
0,art,Best Book Cover Art,https://www.goodreads.com//list/show/101.Best_...,11097
1,art,Inspiring Books,https://www.goodreads.com//list/show/41846.Ins...,4110
2,art,Non-fiction - Something for Everyone,https://www.goodreads.com//list/show/10048.Non...,3778
3,art,Best Cover Art 2011 (Non-YA),https://www.goodreads.com//list/show/14697.Bes...,292
4,art,Children's Picture Books With Beautiful illust...,https://www.goodreads.com//list/show/1191.Chil...,2015
...,...,...,...,...
38484,young-adult,Novels Dealing with Teen Chastity,https://www.goodreads.com//list/show/111561.No...,44
38485,young-adult,ALA 2006 Top Ten Best Books for Young Adults,https://www.goodreads.com//list/show/5191.ALA_...,9
38486,young-adult,Most Boring Book Covers for YA Books,https://www.goodreads.com//list/show/7047.Most...,11
38487,young-adult,Best YA Books in Slang Fiction,https://www.goodreads.com//list/show/15038.Bes...,2


In [197]:
sum(goodreads['count'])

8054706

### request문제로 페이지 오류나면 아래의 코드로 이어서 수집

In [172]:
# 중간에 멈추면 이걸루.. 멈춘 페이지 다음으로 설정하면 됨
drv.get('https://www.goodreads.com/list/tag/young-adult?page=88')

status = True
while status == True:
    soup = BeautifulSoup(drv.page_source, 'html.parser')

    # 장르별 리스트 접근, 한페이지에 최대 30개
    listopia = soup.find('div', {'class':'listRowsFull'}).find_all('div', {'class':'cell'})

    # 해당 리스트에 속하는 개별 타이틀 접근, 포함되는 도서 개수 수집
    cat_href = []
    cat_name = []
    cat_cnt = []

    for i in range(len(listopia)):
        cat_href.append('https://www.goodreads.com/'+listopia[i].find('a', {'class':'listTitle'})['href'])
        cat_name.append(listopia[i].find('a', {'class':'listTitle'}).text)
        cnt = listopia[i].find('div', {'class':'listFullDetails'}).text.strip().replace('\n', '').split('book')[0]
        cat_cnt.append(int(cnt.replace(',', '').strip()))

    gr_data = [gr_name_list[h] for g in range(len(cat_href))]
    data = pd.DataFrame({'genre':gr_data, 'category_list':cat_name, 
                         'href':cat_href, 'count':cat_cnt})
    goodreads = pd.concat([goodreads, data], ignore_index=True)
    
    try:
        if drv.find_element(By.CLASS_NAME, 'next_page.disabled').text == 'next »':
            status = False
            print(f'{gr_name_list[h]} finished~!')
    except:
        drv.find_element(By.CLASS_NAME, 'next_page').click()
        time.sleep(3)

young-adult finished~!


### DB 업로드

In [196]:
user_nm = db_id
passwd = db_pw
host_url = host_url
port_num = port_num
db_name = db_nm
engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
engine_conn = engine.connect()

# insert
goodreads.to_sql(f'goodreads_category_href', con=engine, if_exists='append', index=None)
engine_conn.close()
engine.dispose()